In [1]:
import pandas as pd

playlist_data = pd.read_csv('./spotify_data/playlistCSV.csv', header=0)
streaming_history = pd.read_csv('./spotify_data/streamingHistoryCSV.csv', header=0)

In [2]:
merged_dataset = streaming_history.merge(playlist_data, on='trackName',)
merged_dataset['time'] = merged_dataset['time'].str.split(' ').str[1]
merged_dataset['track_id'] = merged_dataset['trackUri'].str.split('track:').str[1]


In [3]:
def month_to_int(value):
    if value == 'June':
        return 6
    elif value == 'July':
        return 7
    elif value == 'August':
        return 8
    else:
        return 9

In [4]:
merged_dataset['Month'] = merged_dataset['Month'].apply(month_to_int)

In [5]:
merged_dataset['date'] = merged_dataset['Month'].astype(str) + '/' + merged_dataset['Day'].astype(str) + '/' + merged_dataset['Year'].astype(str)
merged_dataset['hour'] = merged_dataset['time'].str.split(':').str[0]
#merged_dataset.to_csv('./cleaned_data/mergedSpotifyData.csv', index=False)

In [6]:
merged_dataset = merged_dataset.drop_duplicates(subset='time')
merged_dataset.head()

,artistName_x,Year,Month,Day,time,Sum of msPlayed,trackName,albumName,artistName_y,trackUri,playlists.name,track_id,date,hour
0,"""Weird Al"" Yankovic",2023,6,22,02:00:00,202920,"Amish Paradise (Parody of ""Gangsta's Paradise""...",Bad Hair Day,"""Weird Al"" Yankovic",spotify:track:5r96TaQquRrlo3Ym3ZlSL2,HOmeWorK BANgers,5r96TaQquRrlo3Ym3ZlSL2,6/22/2023,02
1,"""Weird Al"" Yankovic",2023,7,29,02:38:00,202920,"Amish Paradise (Parody of ""Gangsta's Paradise""...",Bad Hair Day,"""Weird Al"" Yankovic",spotify:track:5r96TaQquRrlo3Ym3ZlSL2,HOmeWorK BANgers,5r96TaQquRrlo3Ym3ZlSL2,7/29/2023,02
2,"""Weird Al"" Yankovic",2023,7,29,01:10:00,328773,"The Saga Begins (Lyrical Adaption of ""American...",Running With Scissors,"""Weird Al"" Yankovic",spotify:track:74sUbOF9Zm8LdGUJjxleTl,HOmeWorK BANgers,74sUbOF9Zm8LdGUJjxleTl,7/29/2023,01
3,...And You Will Know Us by the Trail of Dead,2023,7,23,01:33:00,325600,Wasted State Of Mind,So Divided,...And You Will Know Us by the Trail of Dead,spotify:track:16XbElFPv3A7TrzRJntVYd,Missing Her,16XbElFPv3A7TrzRJntVYd,7/23/2023,01
5,...And You Will Know Us by the Trail of Dead,2023,8,21,03:10:00,25550,Wasted State Of Mind,So Divided,...And You Will Know Us by the Trail of Dead,spotify:track:16XbElFPv3A7TrzRJntVYd,Missing Her,16XbElFPv3A7TrzRJntVYd,8/21/2023,03


In [7]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '3811b236a5dd410e9f8bb2321bbdef9f'
client_secret = '5daa963dee6d41dd820c8d2a776c1fcc'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [12]:
import time
from tqdm import tqdm

def track_info_progress_bar(merged_dataset,api_dataset, start_value, end_value):
    requests_per_hour = 500
    
    requests_made_hour = 0
    start_time = time.time()
    
    

    for track_uri in tqdm(enumerate(merged_dataset['trackUri'][start_value:end_value]), total=end_value-start_value):
        audio_features = sp.audio_features(track_uri[1])
        new_row = pd.DataFrame(audio_features)
        api_dataset = pd.concat([api_dataset,new_row], ignore_index=True)
        
        requests_made_hour += 1
        elapsed_time = time.time() - start_time

       
        if requests_made_hour >= requests_per_hour and elapsed_time < 3600:
            time_to_sleep = 3600 - elapsed_time
            time.sleep(time_to_sleep)
            requests_made_hour = 0
            start_time = time.time()

        #tqdm.write(f"Processed: {index + 1}/{end_value}")
    return api_dataset

In [13]:
start_value = 0
end_value = len(merged_dataset)
api_dataset = pd.DataFrame()
api_dataset = track_info_progress_bar(merged_dataset,api_dataset, start_value, end_value)
display(api_dataset)

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:00<00:00,  8.57it/s]


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.728,0.448,8,-10.540,1,0.1720,0.103,0,0.267,0.483,80.902,audio_features,5r96TaQquRrlo3Ym3ZlSL2,spotify:track:5r96TaQquRrlo3Ym3ZlSL2,https://api.spotify.com/v1/tracks/5r96TaQquRrl...,https://api.spotify.com/v1/audio-analysis/5r96...,202920,4
1,0.728,0.448,8,-10.540,1,0.1720,0.103,0,0.267,0.483,80.902,audio_features,5r96TaQquRrlo3Ym3ZlSL2,spotify:track:5r96TaQquRrlo3Ym3ZlSL2,https://api.spotify.com/v1/tracks/5r96TaQquRrl...,https://api.spotify.com/v1/audio-analysis/5r96...,202920,4
2,0.487,0.429,4,-8.716,0,0.0391,0.332,0,0.207,0.508,143.582,audio_features,74sUbOF9Zm8LdGUJjxleTl,spotify:track:74sUbOF9Zm8LdGUJjxleTl,https://api.spotify.com/v1/tracks/74sUbOF9Zm8L...,https://api.spotify.com/v1/audio-analysis/74sU...,328773,4
3,0.462,0.806,3,-3.283,1,0.0341,0.260,0,0.114,0.354,165.925,audio_features,16XbElFPv3A7TrzRJntVYd,spotify:track:16XbElFPv3A7TrzRJntVYd,https://api.spotify.com/v1/tracks/16XbElFPv3A7...,https://api.spotify.com/v1/audio-analysis/16Xb...,325600,4
4,0.462,0.806,3,-3.283,1,0.0341,0.260,0,0.114,0.354,165.925,audio_features,16XbElFPv3A7TrzRJntVYd,spotify:track:16XbElFPv3A7TrzRJntVYd,https://api.spotify.com/v1/tracks/16XbElFPv3A7...,https://api.spotify.com/v1/audio-analysis/16Xb...,325600,4


In [14]:
columns_to_drop = ['analysis_url','track_href','type','uri']
api_dataset.drop(columns_to_drop, axis=1,inplace=True)
api_dataset.rename(columns={'id': 'track_id'}, inplace=True)


In [15]:
spotify_dataset = merged_dataset.merge(api_dataset, on='track_id')
columns_to_drop = ['trackUri','Sum of msPlayed','artistName_x']
spotify_dataset.drop(columns_to_drop, axis=1,inplace=True)
spotify_dataset.rename(columns={'artistName_y': 'artistName'}, inplace=True)
spotify_dataset.to_csv('apiData.csv', index=False)